In [30]:
import pandas as pd
import json

In [31]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [50]:
# similariy between two strings
fuzz.ratio("Where can I begin.", \
    "– Where can I begin.")


95

##### Mistakes of correctness.csv
###### 1. Some pred_target's format and length are incorrect due to wrong conversion
###### 2. 'sent_id' of 'graph-test-correctness.csv' are all wrong due to wrong sorting process
###### 3. It's necessary to adjust prediction results (json files) manually in certain steps 

In [37]:
df = pd.read_csv('../semEval16/check_correctness/graph-test-correctness.csv', encoding='utf-8')
df[df.Target_matching == True]

,sent_id,text,Source,Target,Polar_expression,Polarity,pred_target correct,pred_expression correct,hasTarget,hasExpression,Target_matching
52,en_BlueRibbonSushi_478218171:1,"Best food, phenominal service",NaN,service,Best,Positive,T,T,T,T,True
53,en_BlueRibbonSushi_478218171:2,That was the last night in Vegas and was too l...,NaN,most of the restaurant,closing -,Positive,T,T,T,T,True
54,en_BlueRibbonSushi_478218171:2,Best Neighborhood Standby.,NaN,Neighborhood Standby.,Best,Positive,T,T,T,T,True
55,en_BlueRibbonSushi_478218171:3,Open & Cool Place with the Best Pizza and Coffee,NaN,Coffee,Best,Positive,T,T,T,T,True
56,en_BlueRibbonSushi_478218333:0,Mediocre food,NaN,food,Mediocre,Positive,T,T,T,T,True
...,...,...,...,...,...,...,...,...,...,...,...
388,en_StackRestaurant__Bar_478538239:19,Try the Chef's Choice for sushi as the smoked ...,NaN,the rolls,tasty.,Positive,T,T,T,T,True
389,en_StackRestaurant__Bar_478538239:19,The pizza is delicious and the salads are fant...,NaN,the salads,fantastic.,Positive,T,T,T,T,True
390,en_StackRestaurant__Bar_478538239:7,The wine list is incredible and extensive and ...,NaN,the staff,"very nice, good",Positive,T,T,T,T,True
391,en_StackRestaurant__Bar_478538239:9,Best Indian food in L.A.,NaN,food,Best,Positive,T,T,T,T,True


In [38]:
# open training data or test data
with open('../semEval16/graph_parser/predictions/test-gold/opener_en_prediction.json') as f:
    json_data = json.load(f)

json_data[:10]

[{'sent_id': 'en_BlueRibbonSushi_478218171:0', 'text': 'Yum!', 'opinions': []},
 {'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard'}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard'},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard'}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source': [[], []],
    'Target': [['Green Tea creme brulee'], ['0:22']],


In [51]:
# adjust sent_id from test-correctness
# run this cell only for test data

c = 0
for i in range(len(df)):
    flag = False

    for j in json_data:
        if df.iloc[i]['text'] == j['text'] or fuzz.ratio(df.iloc[i]['text'], j['text']) >= 95:
            c += 1
            flag = True
            df.iloc[i,0] = j['sent_id'] # edit sent_id
            break
            
    if flag == False:
        print(i, df.iloc[i]['text'])

print(f'number of df rows with correct sent_id: {c}')

803 #NAME?
number of df rows with correct sent_id: 847


In [52]:
df

,sent_id,text,Source,Target,Polar_expression,Polarity,pred_target correct,pred_expression correct,hasTarget,hasExpression,Target_matching
0,en_MiopostoCaffe_479702043:4,The pizzas are thin crust and the menu offers ...,NaN,the menu offers,toppings.,Positive,T,F,T,T,False
1,en_SchoonerOrLater_479842021:6,As it turns out the owner was seated right nex...,NaN,on our problems,very dismissive,Negative,F,T,T,T,False
2,en_India'sGrill_477960693:0,Best Indian food in L.A.,NaN,food,Indian,Positive,T,F,T,T,False
3,en_MercedesRestaurant_478010602:0,"Great food, better Margaritas!",NaN,"food, better Margaritas!",Great,Positive,F,T,T,T,False
4,en_Murphy's_478075037:3,The regular menu here is slightly above averag...,NaN,The regular menu,average,Negative,T,F,T,T,False
...,...,...,...,...,...,...,...,...,...,...,...
843,en_StackRestaurant__Bar_478538242:0,Seafood Plus,NaN,NaN,NaN,NaN,NaN,NaN,F,F,False
844,en_StackRestaurant__Bar_478538242:1,This was great dining experience.,NaN,NaN,great,Positive,NaN,NaN,F,T,False
845,en_StackRestaurant__Bar_478538242:2,If you're going to see Danny Gans or just stay...,NaN,NaN,NaN,NaN,NaN,NaN,F,F,False
846,en_StackRestaurant__Bar_478538242:4,Seabass on lobster risotto was the best.,NaN,NaN,NaN,NaN,NaN,NaN,F,F,False


In [58]:
# assign 'Matching' to json based on correctness.csv(df)

count = 0

for i in json_data:
    sid = i['sent_id'] 

    for j in i['opinions']:
        j['Matching'] = False

        if len(j['Target'][0]) > 0:
            tar = j['Target'][0][0]
            exp = j['Polar_expression'][0][0]
            df2 = df[df.sent_id == sid]
            # print(df2)

            for k in range(len(df2)):
                # match pairs of json and csv
                if (pd.isnull(df2.iloc[k]['Target']) == False) and (df2.iloc[k]['Target'] == tar) and (df2.iloc[k]['Polar_expression'] == exp):
                    # print(df2.iloc[k]['Target'], df2.iloc[k]['Polar_expression'])   
                    if (df2.iloc[k]['Target_matching'] == True):
                        j['Matching'] = True
                        count += 1
                        break
                            
        
print(f'number of true targets: {count}')

number of true targets: 336


In [55]:
# deep copy original json file
import copy
json_list = copy.deepcopy(json_data)

In [59]:
# check true pairs in json 
true_df = df[df.Target_matching == True]

wrong = 0
sid_true = []
for s in range(len(true_df)):
    flag = False

    for t in range(len(json_list)):
        for r in json_list[t]['opinions']:
            if (r['Matching']) and (true_df.iloc[s]['sent_id'] == json_list[t]['sent_id']):
                if (true_df.iloc[s]['Target'] == r['Target'][0][0]) and (true_df.iloc[s]['Polar_expression'] == r['Polar_expression'][0][0]):
                    flag = True
                    break
        if flag == True:
            break


    if flag == False:
        wrong += 1
        sid_true.append(true_df.iloc[s]['sent_id'])
        print(true_df.iloc[s]['sent_id'] + ', ' + true_df.iloc[s]['Target'] + ', ' + true_df.iloc[s]['Polar_expression'])

print(f'number of missing pairs: {wrong}')

en_MiopostoCaffe_479702043:10, The pizzas, not huge
en_MiopostoCaffe_479702043:4, The pizzas, thin crust
en_MiopostoCaffe_479702043:6, The pizzas, light
en_MiopostoCaffe_479702043:6, The pizzas, scrumptious.
number of missing pairs: 4


In [60]:
# correct 'Matching' results manually
for i in range(len(json_list)):
    if json_list[i]['sent_id'] in sid_true:
        for j in range(len(json_list[i]['opinions'])):
            print(i,j,json_list[i]['opinions'][j])

json_list[557]['opinions'][1]['Matching'] = True
json_list[559]['opinions'][0]['Matching'] = True
json_list[559]['opinions'][1]['Matching'] = True
json_list[563]['opinions'][0]['Matching'] = True

557 0 {'Source': [[], []], 'Target': [['the menu offers'], ['31:46']], 'Polar_expression': [['toppings.'], ['78:87']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
557 1 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['thin crust'], ['16:26']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
559 0 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['scrumptious.'], ['26:38']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
559 1 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['light'], ['16:21']], 'Polarity': 'Positive', 'Intensity': 'Standard', 'Matching': False}
563 0 {'Source': [[], []], 'Target': [['The pizza’s'], ['0:11']], 'Polar_expression': [['not huge'], ['16:24']], 'Polarity': 'Negative', 'Intensity': 'Standard', 'Matching': False}


In [61]:
# check numbe of true pairs

count = 0
for i in range(len(json_list)):
        for j in json_list[i]['opinions']:
            if j['Matching']:
                count += 1

print(f'number of true targets: {count}')

number of true targets: 340


In [62]:
# remove all false pairs

for i in range(len(json_list)):
    new_list = [j for j in json_list[i]['opinions'] if j['Matching']]
    json_list[i]['opinions'] = new_list
        
json_list

[{'sent_id': 'en_BlueRibbonSushi_478218171:0', 'text': 'Yum!', 'opinions': []},
 {'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True}]},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source'

In [64]:
# remove all empty opinions
json_new = []

for i in range(len(json_list)):
    if len(json_list[i]['opinions']) > 0:
        json_new.append(json_list[i])

print(f'number of text: {len(json_new)}')


number of text: 249


In [65]:
# add key of dataset

for i in range(len(json_new)):
    json_new[i]['dataset'] = 's16+_test'

json_new

[{'sent_id': 'en_BlueRibbonSushi_478218171:1',
  'text': 'Serves really good sushi.',
  'opinions': [{'Source': [[], []],
    'Target': [['sushi.'], ['19:25']],
    'Polar_expression': [['good'], ['14:18']],
    'Polarity': 'Positive',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_test'},
 {'sent_id': 'en_BlueRibbonSushi_478218171:2',
  'text': 'Not the biggest portions but adequate.',
  'opinions': [{'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['Not the biggest'], ['0:15']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True},
   {'Source': [[], []],
    'Target': [['portions'], ['16:24']],
    'Polar_expression': [['adequate.'], ['29:38']],
    'Polarity': 'Negative',
    'Intensity': 'Standard',
    'Matching': True}],
  'dataset': 's16+_test'},
 {'sent_id': 'en_BlueRibbonSushi_478218171:3',
  'text': 'Green Tea creme brulee is a must!',
  'opinions': [{'Source': [[], []],
    'Target': [[

In [66]:
# convert to json file

with open('../semEval16_plus/s16+_test.json', 'w') as outfile:
    json.dump(json_new, outfile)